In [200]:
import pandas as pd
import json
import re
import os

In [245]:
food_type_cols = ['2.您的美食属于哪个菜系/地区？:川菜（四川，重庆）', '2.您的美食属于哪个菜系/地区？:湘菜（湖南）',
       '2.您的美食属于哪个菜系/地区？:鲁菜（山东）', '2.您的美食属于哪个菜系/地区？:西北菜 （陕西，甘肃等地）',
       '2.您的美食属于哪个菜系/地区？:赣菜（江西）', '2.您的美食属于哪个菜系/地区？:东北菜 （黑龙江等地）',
       '2.您的美食属于哪个菜系/地区？:黔菜 (贵州）', '2.您的美食属于哪个菜系/地区？:内蒙菜 （内蒙古）',
       '2.您的美食属于哪个菜系/地区？:徽菜 （安徽）', '2.您的美食属于哪个菜系/地区？:粤菜（广东等地）',
       '2.您的美食属于哪个菜系/地区？:苏菜（江苏）', '2.您的美食属于哪个菜系/地区？:浙菜（浙江）',
       '2.您的美食属于哪个菜系/地区？:闽菜（福建）', '2.您的美食属于哪个菜系/地区？:新疆菜',
       '2.您的美食属于哪个菜系/地区？:其他____{fillblank-f468}',
       '2.您的美食属于哪个菜系/地区？:其他____{fillblank-f468}[选项填空]']

def extract_img_file_name(row):
    hyperlink = row["1.请上传您的美食照片_附件1"]
    pattern = r'"([^"]+)"\)$'
    match = re.search(pattern, hyperlink)

    if match:
        image_name = match.group(1)
        return str(row['编号'])+ "_" + image_name
    else:
        return None
    
def clean_item(row):
    item_dict = {"main_ingredient":set()}
    row_dict = row.dropna().to_dict()
    keep_cols = {
        '编号': "id",
        '美食名称': "food_name",
        '4.您的美食来自': "food_location"
    }
    for key in row_dict.keys():
        if key in food_type_cols:
            if len(row_dict[key].split(".")) == 2:
                item_dict["food_type"] = row_dict[key].split(".")[1]
            else:
                item_dict["food_type"] = row_dict[key]
        if key in keep_cols:
            if key == "4.您的美食来自":
                if len(row_dict[key].split(".")) == 2:
                    item_dict[keep_cols[key]] = row_dict[key].split(".")[1] 
            else:
                item_dict[keep_cols[key]] = row_dict[key]
    item_dict["food_file"] = extract_img_file_name(row)
    
    main_ingredients = ["鱼", "肉", "牛", "鸡", "羊", "蛋", "虾", "肉", "饭", "蟹", "豆腐", "面", "鸭", "鹅"]
    for ingredient in main_ingredients:
        if ingredient in item_dict["food_name"]:
            item_dict["main_ingredient"].add(ingredient)
    item_dict["main_ingredient"] = list(item_dict["main_ingredient"])
    if len(item_dict["main_ingredient"]) == 0:
        item_dict["main_ingredient"] = ["其他"]
    return item_dict

In [262]:
def clean_image_collections(survey_file):
    survey_folder_dict = {
        "14456664_202404221743366663_verified.csv": "14456664_all_202404292352223293",
        "14521898_202404291705453365_verified.csv": "14521898_all_202404291706016981"       
        }
    survey_df = pd.read_csv(survey_file, delimiter=";")
    verified_df = survey_df[survey_df['Image-confirmed'] == "Good"]
    food_dict = verified_df.apply(clean_item, axis=1).to_list()
    for item in food_dict:
        if item["food_file"]:
            item["food_file"] = f"{survey_folder_dict[os.path.basename(survey_file)]}/{item['food_file']}"
    with open(survey_file.replace(".csv", "_cleaned.json"), "w") as f:
        json.dump(food_dict, f, ensure_ascii=False, indent=4)
    return food_dict

In [247]:
survey1_file = "/Users/wli/projects/foodie-dataset/data-collection/14456664_202404221743366663_verified.csv"
survey2_file = "/Users/wli/projects/foodie-dataset/data-collection/14521898_202404291705453365_verified.csv"

In [248]:
survey1_dict = clean_image_collections(survey1_file)

In [249]:
survey2_dict = clean_image_collections(survey2_file)

In [250]:
len(survey1_dict), len(survey2_dict)

(175, 140)

In [252]:
for item in survey1_dict:
    if item["main_ingredient"] == ["牛"]:
        print(item)

{'main_ingredient': ['牛'], 'id': 37, 'food_name': '牛油火锅', 'food_type': '川菜（四川，重庆）', 'food_location': '餐馆', 'food_file': '14456664_all_202404292352223293/37_IMG_1059.jpeg'}
{'main_ingredient': ['牛'], 'id': 49, 'food_name': '干锅牛蛙', 'food_type': '川菜（四川，重庆）', 'food_location': '餐馆', 'food_file': '14456664_all_202404292352223293/49_IMG_1004.jpeg'}
{'main_ingredient': ['牛'], 'id': 152, 'food_name': '干炒牛河', 'food_type': '粤菜（广东等地）', 'food_location': '餐馆', 'food_file': '14456664_all_202404292352223293/152_IMG_20240414_124846.jpg'}
{'main_ingredient': ['牛'], 'id': 174, 'food_name': '招牌牛蛙', 'food_type': '湘菜（湖南）', 'food_location': '餐馆', 'food_file': '14456664_all_202404292352223293/174_IMG_1449.jpeg'}
{'main_ingredient': ['牛'], 'id': 222, 'food_name': '一品干香牛蛙', 'food_location': '餐馆', 'food_file': '14456664_all_202404292352223293/222_1d59bbd9409b075e106f87dd5d58b559.jpeg'}


In [269]:
def extract_overlapped_phrases(data, phrase_field):
    # Extract the phrases from the list of dictionaries
    phrases = [item[phrase_field] for item in data]
    
    # Sort the phrases by length in descending order
    phrases = sorted(phrases, key=len, reverse=True)
    
    overlapped_phrases = []
    
    for i in range(len(phrases)):
        for j in range(i+1, len(phrases)):
            # Check if the current phrase is a substring of any other phrase
            if phrases[j] in phrases[i]:
                overlapped_phrases.append(phrases[j])
                break
    
    # Group the list of dictionaries by the overlapped phrases
    grouped_data = {}
    for item in data:
        for phrase in overlapped_phrases:
            if phrase in item[phrase_field]:
                if phrase not in grouped_data:
                    grouped_data[phrase] = []
                grouped_data[phrase].append(item)
                break
    
    return grouped_data

## extract food groups

In [270]:

result = extract_overlapped_phrases(survey1_dict, "food_name")
for phrase, items in result.items():
    print(f"Phrase: {phrase}")
    for item in items:
        print(f"- {item}")
    print()

Phrase: 大盘鸡
- {'main_ingredient': ['鸡'], 'id': 3, 'food_name': '大盘鸡', 'food_location': '家里自制', 'food_file': '14456664_all_202404292352223293/3_67F2793A-58D3-4845-9515-6FD3734B0311.jpeg'}

Phrase: 辣子鸡
- {'main_ingredient': ['鸡'], 'id': 4, 'food_name': '辣子鸡', 'food_location': '家里自制', 'food_file': '14456664_all_202404292352223293/4_BDA2BB3D-53D1-46CA-B83B-D8D5280F0754.jpeg'}

Phrase: 酸菜鱼
- {'main_ingredient': ['鱼'], 'id': 13, 'food_name': '酸菜鱼', 'food_type': '川菜（四川，重庆）', 'food_location': '家里自制', 'food_file': '14456664_all_202404292352223293/13_IMG_20230522_201922.jpg'}
- {'main_ingredient': ['鱼'], 'id': 72, 'food_name': '酸菜鱼', 'food_type': '川菜（四川，重庆）', 'food_location': '餐馆', 'food_file': '14456664_all_202404292352223293/72_IMG_0618.jpeg'}

Phrase: 海鲜
- {'main_ingredient': ['其他'], 'id': 16, 'food_name': '捞汁小海鲜', 'food_type': '闽菜（福建）', 'food_location': '餐馆', 'food_file': '14456664_all_202404292352223293/16_IMG_0704.jpeg'}
- {'main_ingredient': ['饭'], 'id': 91, 'food_name': '海鲜炒饭', 'food_typ